In [2]:
# run dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

/Users/madelynbaker/Downloads/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Import dataframes that give features

In [4]:
#import dataset that you will be predicting upon (Target = predicting a win, draw, or loss)
#But also, the match data needs to be organized differently.
df_matches = pd.read_csv("Features/wc_matches.csv")
df_matches

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference,year
0,1314,1930-07-13,Belgium,United States,0,3,FIFA World Cup,Montevideo,Uruguay,True,United States,3,1930
1,1315,1930-07-13,France,Mexico,4,1,FIFA World Cup,Montevideo,Uruguay,True,France,3,1930
2,1316,1930-07-14,Brazil,Yugoslavia,1,2,FIFA World Cup,Montevideo,Uruguay,True,Yugoslavia,1,1930
3,1318,1930-07-15,Argentina,France,1,0,FIFA World Cup,Montevideo,Uruguay,True,Argentina,1,1930
4,1319,1930-07-16,Chile,Mexico,3,0,FIFA World Cup,Montevideo,Uruguay,True,Chile,3,1930
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,43441,2022-03-30,Costa Rica,United States,2,0,FIFA World Cup qualification,San José,Costa Rica,False,Costa Rica,2,2022
4178,43442,2022-03-30,Panama,Canada,1,0,FIFA World Cup qualification,Panama City,Panama,False,Panama,1,2022
4179,43538,2022-06-05,Wales,Ukraine,1,0,FIFA World Cup qualification,Cardiff,Wales,False,Wales,1,2022
4180,43704,2022-06-13,Australia,Peru,0,0,FIFA World Cup qualification,Al Rayyan,Qatar,True,draw,0,2022


In [5]:
#limit the dataframe rows to the relevant teams playing in the 2022 world cup
teams_2022 = ['Qatar', 'Netherlands', 'Senegal', 'Ecuador', 
            'England', 'United States', 'Wales', 'Iran', 
            'Argentina', 'Poland', 'Mexico', 'Saudi Arabia', 
            'France', 'Denmark', 'Tunisia', 'Australia', 
            'Germany', 'Spain', 'Japan', 'Costa Rica', 
            'Belgium', 'Croatia', 'Canada', 'Morocco', 
            'Brazil', 'Switzerland', 'Serbia', 'Cameroon', 
            'Portugal', 'Uruguay', 'Ghana', 'Korea Republic']

In [6]:
#call all csv files that were created (feature files)
rank = pd.read_csv("Features/fifa_rank.csv")
player_avg = pd.read_csv("Features/player_averages_for_teams.csv")
win_ratio = pd.read_csv("Features/ratio_played_vs_won.csv")
delta_pts = pd.read_csv("Features/total_score_margins.csv")
games = pd.read_csv("Features/adv_df_pivot.csv")

In [7]:
rank = rank.drop(['Unnamed: 0', 'Points'], axis=1)
rank.head()

,Rank,Country
0,1,Brazil
1,2,Belgium
2,3,Argentina
3,4,France
4,5,England


In [8]:
player_avg= player_avg.rename(columns={"Nationality": "Country"})
player_avg.head()

,Country,Overall,Potential,Height,Weight,Age,WeakFoot,SkillMoves,Value,Wage,...,LongShots,Curve,FKAcc,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes
0,Afghanistan,65.500000,68.000000,172.500000,65.000000,25.500000,3.000000,2.500000,9.125000e+05,1950.000000,...,43.500000,53.000000,51.500000,54.000000,52.500000,11.000000,13.000000,6.500000,8.500000,10.500000
1,Albania,65.000000,70.620000,182.480000,75.860000,25.120000,2.940000,2.400000,2.601900e+06,8024.000000,...,49.340000,46.760000,44.380000,50.340000,44.700000,13.500000,13.500000,12.760000,13.240000,13.640000
2,Algeria,70.462963,73.166667,182.018519,75.351852,27.296296,2.944444,2.759259,5.246296e+06,21177.777778,...,50.166667,53.277778,48.796296,53.740741,47.481481,16.981481,17.148148,16.777778,16.222222,17.703704
3,Andorra,64.000000,64.000000,184.000000,79.000000,30.000000,3.000000,2.000000,4.750000e+05,1500.000000,...,32.000000,37.000000,29.000000,45.000000,41.000000,8.000000,9.000000,7.000000,12.000000,6.000000
4,Angola,67.095238,73.857143,178.523810,73.047619,24.809524,3.000000,2.809524,2.574048e+06,7066.666667,...,52.190476,54.428571,47.000000,53.666667,51.285714,13.142857,13.857143,12.238095,12.809524,12.285714


In [9]:
win_ratio = win_ratio.drop(['Unnamed: 0'], axis=1)
win_ratio.head()

,Country,total_wins,total_games,ratio_won
0,Argentina,39,65.0,0.600000
1,Australia,2,16.0,0.125000
2,Belgium,21,42.0,0.500000
3,Brazil,56,77.0,0.727273
4,Cameroon,4,23.0,0.173913


In [10]:
delta_pts = delta_pts.drop(['Unnamed: 0'], axis=1)
delta_pts.head()

,Country,win_margins,lose_margins
0,Argentina,2.823529,-2.059524
1,Belgium,2.500000,-1.903846
2,Brazil,3.790698,-1.800000
3,Cameroon,2.000000,-3.077778
4,Costa Rica,2.000000,-2.571429


In [11]:
games= games.rename(columns={"country": "Country"})
games.head()

,Country,final,group stage,quarter-finals,round of 16,second group stage,semi-finals,third-place match
0,Argentina,4.0,36.0,6.0,8.0,8.0,3.0,0.0
1,Australia,0.0,15.0,0.0,1.0,0.0,0.0,0.0
2,Belgium,0.0,27.0,3.0,6.0,2.0,2.0,2.0
3,Brazil,4.0,39.0,9.0,9.0,8.0,5.0,3.0
4,Cameroon,0.0,21.0,1.0,1.0,0.0,0.0,0.0


In [12]:
# merge dataframes on the column "Country"

In [13]:
from functools import reduce
dfs = [rank, player_avg, win_ratio, delta_pts, games] # list of dataframes

compiled_df = reduce(lambda  left,right: pd.merge(left,right,on=['Country'],
                                            how='outer'), dfs)
compiled_df

,Rank,Country,Overall,Potential,Height,Weight,Age,WeakFoot,SkillMoves,Value,...,ratio_won,win_margins,lose_margins,final,group stage,quarter-finals,round of 16,second group stage,semi-finals,third-place match
0,1.0,Brazil,70.967232,73.102825,180.677966,75.772881,26.162712,2.966102,2.554802,5.522758e+06,...,0.727273,3.790698,-1.800000,4.0,39.0,9.0,9.0,8.0,5.0,3.0
1,2.0,Belgium,67.676667,73.903333,182.300000,75.126667,24.496667,3.173333,2.416667,4.685353e+06,...,0.500000,2.500000,-1.903846,0.0,27.0,3.0,6.0,2.0,2.0,2.0
2,3.0,Argentina,67.806316,72.728421,178.816842,74.710526,26.442105,2.916842,2.473684,3.400971e+06,...,0.600000,2.823529,-2.059524,4.0,36.0,6.0,8.0,8.0,3.0,0.0
3,4.0,France,67.238569,73.587475,181.771372,75.247515,24.638171,2.895626,2.421471,4.636357e+06,...,0.580000,3.576190,-1.400000,3.0,27.0,5.0,5.0,2.0,5.0,2.0
4,5.0,England,63.272833,70.315300,181.833042,74.194881,24.467132,2.891798,2.279814,2.334097e+06,...,0.733333,3.232143,-1.064706,0.0,30.0,6.0,7.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,NaN,United Arab Emirates,63.560000,67.400000,176.400000,69.440000,26.800000,3.040000,2.200000,9.204762e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,NaN,Uzbekistan,64.625000,70.750000,179.625000,72.125000,24.625000,3.375000,2.375000,1.153125e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,NaN,Venezuela,61.721393,67.099502,177.746269,72.741294,25.189055,2.751244,2.268657,1.131020e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,NaN,Zambia,67.666667,74.000000,179.888889,75.111111,23.666667,3.222222,2.777778,3.861111e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pd.set_option('display.max_columns', None)

In [15]:
compiled_df = compiled_df[(compiled_df['Country'].isin(teams_2022))]
compiled_df

,Rank,Country,Overall,Potential,Height,Weight,Age,WeakFoot,SkillMoves,Value,Wage,Nation_KitNumber,Club_KitNumber,BallControl,Dribbling,Marking,SlideTackle,StandTackle,Aggression,Reactions,AttPosition,Interceptions,Vision,Composure,Crossing,ShortPass,LongPass,Acceleration,Stamina,Strength,Balance,SprintSpeed,Agility,Jumping,Heading,ShotPower,Finishing,LongShots,Curve,FKAcc,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes,total_wins,total_games,ratio_won,win_margins,lose_margins,final,group stage,quarter-finals,round of 16,second group stage,semi-finals,third-place match
0,1.0,Brazil,70.967232,73.102825,180.677966,75.772881,26.162712,2.966102,2.554802,5.522758e+06,15841.109853,12.0,21.279096,64.129944,61.331073,49.257627,49.161582,51.312994,60.401130,68.138983,55.312994,50.683616,58.764972,64.836158,55.091525,63.977401,57.731073,66.393220,65.938983,67.662147,63.723164,66.449718,65.471186,65.855367,57.781921,63.784181,51.201130,53.818079,52.977401,49.543503,54.741243,49.570621,15.854237,16.118644,15.724294,15.892655,16.342373,56.0,77.0,0.727273,3.790698,-1.800000,4.0,39.0,9.0,9.0,8.0,5.0,3.0
1,2.0,Belgium,67.676667,73.903333,182.300000,75.126667,24.496667,3.173333,2.416667,4.685353e+06,14919.666667,12.0,25.203333,60.603333,57.073333,45.990000,46.456667,48.223333,55.353333,62.580000,49.653333,46.986667,55.933333,60.656667,52.840000,60.726667,56.400000,63.823333,60.303333,64.023333,64.216667,63.550000,64.530000,64.966667,50.426667,60.153333,45.413333,48.213333,48.943333,45.296667,48.043333,44.650000,18.266667,18.416667,18.183333,18.086667,18.506667,21.0,42.0,0.500000,2.500000,-1.903846,0.0,27.0,3.0,6.0,2.0,2.0,2.0
2,3.0,Argentina,67.806316,72.728421,178.816842,74.710526,26.442105,2.916842,2.473684,3.400971e+06,10585.157895,12.0,17.023158,60.943158,58.431579,47.120000,46.127368,47.711579,57.486316,63.905263,52.948421,46.431579,55.849474,60.892632,51.175789,60.735789,54.655789,64.472632,63.560000,64.737895,64.950526,64.333684,64.454737,65.637895,54.482105,61.314737,48.567368,49.886316,49.624211,44.330526,51.426316,45.255789,16.545263,16.392632,16.363158,16.362105,16.547368,39.0,65.0,0.600000,2.823529,-2.059524,4.0,36.0,6.0,8.0,8.0,3.0,0.0
3,4.0,France,67.238569,73.587475,181.771372,75.247515,24.638171,2.895626,2.421471,4.636357e+06,13453.976143,12.0,19.766402,61.004970,58.146123,47.757455,48.030815,49.921471,56.703777,62.412525,51.207753,48.309145,55.052684,60.042744,51.854871,61.669980,55.352883,64.415507,62.129225,65.846918,63.930417,64.401590,63.178926,64.753479,54.634195,59.320080,45.987078,47.150099,49.329026,43.612326,49.811133,43.576541,15.719682,15.820080,15.544732,15.483101,16.099404,29.0,50.0,0.580000,3.576190,-1.400000,3.0,27.0,5.0,5.0,2.0,5.0,2.0
4,5.0,England,63.272833,70.315300,181.833042,74.194881,24.467132,2.891798,2.279814,2.334097e+06,9738.917976,12.0,18.353694,56.256545,53.958697,46.172193,45.197789,47.277487,54.671321,59.031995,48.528214,45.464223,52.079116,56.420593,47.621873,55.929610,50.592205,64.887144,62.584642,64.004072,64.641070,64.969168,63.536358,64.731239,50.419430,54.691099,44.261198,44.397324,45.015707,40.426992,46.616638,40.319372,15.452007,15.757999,15.696917,15.456079,15.900524,22.0,30.0,0.733333,3.232143,-1.064706,0.0,30.0,6.0,7.0,2.0,2.0,2.0
5,6.0,Spain,69.285189,74.675253,180.231831,73.692732,25.685373,3.055198,2.573137,5.212190e+06,14916.743330,12.0,17.748850,62.510580,58.661454,49.388224,47.719411,49.743330,55.850966,65.594296,53.728611,49.026679,57.663293,61.252990,54.154554,63.374425,57.041398,64.237351,62.781969,63.208832,65.416743,64.143514,64.206072,64.022999,53.447102,61.344986,49.776449,50.751610,52.478381,46.367065,50.922723,44.752530,16.856486,17.188592,16.993560,16.811408,17.266789,25.0,30.0,0.833333,3.733766,-1.241071,1.0,33.0,4.0,7.0,2.0,1.0,0.0
6,8.0,Netherlands,67.073913,73.165217,183.026087,75.917391,24.436957,2.989130,2.397826,4.045946e+06,10634.565217,12.0,19.236957,59.904348,56.486957,47.480435,46.786957,48.602174,56.771739,63.328261,49.352174,47.228261

In [16]:
#remove data that we consider irrelevant to predicting the feature
df_matches = df_matches.drop(['date', 'Unnamed: 0', 'home_score', 'away_score', 'tournament', 'city', 'country', 'neutral','year'], axis=1)

In [17]:
#if we are going to split the data per country, we will need to define the match data below differently:
#each row is one of the 32 countries
#previous match data would be a column
#ie. Belgium on row, and United States, Mexico, Yugoslavia, etc on y axis
#not exactly sure if this will work

In [18]:
#define home team, away team, and winning team
#The winning team column will show "2" if the home team has won, "1" if it was a tie, and "0" if the away team has won
#The model will be build to predict the "winning_team"

df_matches = df_matches.reset_index(drop=True)
df_matches.loc[df_matches.winning_team == df_matches.home_team,'winning_team']=2
df_matches.loc[df_matches.winning_team == 'draw', 'winning_team']=1
df_matches.loc[df_matches.winning_team == df_matches.away_team, 'winning_team']=0

df_matches

,home_team,away_team,winning_team,goal_difference
0,Belgium,United States,0,3
1,France,Mexico,2,3
2,Brazil,Yugoslavia,0,1
3,Argentina,France,2,1
4,Chile,Mexico,2,3
...,...,...,...,...
4177,Costa Rica,United States,2,2
4178,Panama,Canada,2,1
4179,Wales,Ukraine,2,1
4180,Australia,Peru,1,0


# Running (up that hill) the model

In [19]:
final = pd.get_dummies(df_matches, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Separate X and y sets
X = final.drop(['winning_team'], axis=1)
y = final["winning_team"]
y=y.astype('int')

# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
#classify data as 0, 1, or 2 (home team lost, drew, or won)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

score = classifier.score(X_train, y_train)
score2 = classifier.score(X_test, y_test)
print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

In [ ]:
schedule = pd.read_csv('Data/schedule.csv')

In [ ]:
rank = pd.read_csv("Features/fifa_rank.csv")
rank

In [ ]:
# Create new columns with ranking position of each team
schedule.insert(1, 'first_position', schedule['Home Team'].map(rank.set_index('Country')['Rank']))
schedule.insert(2, 'second_position', schedule['Away Team'].map(rank.set_index('Country')['Rank']))

# We only need the group stage games, so we have to slice the dataset
schedule = schedule.iloc[:48, :]
schedule

In [ ]:
pred_set= []
for index, row in schedule.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'home_team': row['Home Team'], 'away_team': row['Away Team'], 'winning_team': None})
    else:
        pred_set.append({'home_team': row['Away Team'], 'away_team': row['Home Team'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set

pred_set.head()

In [ ]:
# # Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]

# Remove winning team column
pred_set = pred_set.drop(['winning_team'], axis=1)

pred_set.head()

In [ ]:
#group matches
predictions = classifier.predict(pred_set)
for i in range(schedule.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 2:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    elif predictions[i] == 1:
        print("Draw")
    elif predictions[i] == 0:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ', '%.3f'%(classifier.predict_proba(pred_set)[i][2]))
    print('Probability of Draw: ', '%.3f'%(classifier.predict_proba(pred_set)[i][1]))
    print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(classifier.predict_proba(pred_set)[i][0]))
    print("")